## Generate N random points in $[0,1]^2$

In [6]:
N = 200 ## number of points
d = 2 ## dimension of the points
Pts = rand(N, d); ## the points (coordinates as columns)

## plot for illustration
using Plots
plotly()
scatter(Pts[:,1], Pts[:,2], label = "Points")

## Set the hyperparameters

In [4]:
## Set global hyperparameters
alp = 1 ## hyperparameter alpha
bet = 1 ## hyperparameter beta
rho = 0.3 ## evaporation rate of pheromone
Q = 10 ## total pheromone for an ant
nAnts = 100 ## number of ants

## Set the starting point and goal
Origin = 22 ## the starting point
Goal = 8; ## the goal

In [7]:
## plot for illustration
scatter!([Pts[Origin,1]], [Pts[Origin,2]], label = "Origin")
scatter!([Pts[Goal,1]], [Pts[Goal,2]], label = "Goal")

In [4]:
## demonstration of sampling with assigned weights (probabilities)
using StatsBase
w = aweights([0.2, 0.1, 0.3])
sample([0,1,2],w)

2

In [8]:
## compute the distance matrix
using LinearAlgebra ## for the function "norm"
Dist = [norm(Pts[i,:].-Pts[j,:]) for i=1:N, j=1:N]; ## the distance matrix of the points

In [9]:
## Dictionary for collecting the best result in each iteration
Iteration_Dict = Dict()

Dict{Any,Any} with 0 entries

In [10]:
using StatsBase ## for function "sample"
N_I = 100 ## number of iterations
for I = 1:N_I
    Tau = (1/N).*ones(N,N); ## the prior transition probability
    Best_L = Inf ## the best (shortest) length
    BestPath = 0 ## the best (shortest) path
    
    L = 0 ## variable for stopping criterion
    L_previous = Inf ## variable for stopping criterion
    
    Final_path = 0
    Final_L = Inf
    
    while (abs(L-L_previous)>1.0e-7)
        L_previous = copy(L)
        Cand = setdiff(collect(1:N),union([Origin],[Goal])) ## the candidates
        cp = copy(Origin) ## cp =  current point
        path_a = Int[Origin]
        
        for p = 2:N-1
            w = aweights((Tau[cp,Cand].^alp).*((Dist[cp,Cand].^(-1)).^bet))
            cp = sample(Cand,w)
            Cand = setdiff(Cand,[cp])
            push!(path_a,cp)
        end
        push!(path_a,Goal)
        path_a
        Final_path = path_a
        L = sum([Dist[path_a[i],path_a[i+1]] for i=1:N-1]) ## total length of path_a
        
        ## println(L)
        
        ## store the best record
        if L<Best_L
            Best_L = L
            BestPath = path_a
        end
    
        Tau = Tau*(1-rho) ## evaporation update
        for i=1:N-1
            Tau[path_a[i], path_a[i+1]] = Tau[path_a[i], path_a[i+1]] + Q/L ## Pheromone update
        end
        Tau
    end
        
    Iteration_Dict[I] = (Best_L, BestPath)
end

In [8]:
sortslices([[Iteration_Dict[k][1] for k = 1:N_I] collect(1:N_I)],dims = 1)

100×2 Array{Float64,2}:
 28.3234  32.0
 29.4383  23.0
 30.2105  52.0
 30.671   93.0
 30.7762  33.0
 31.2751  38.0
 31.3696  13.0
 31.4611  99.0
 31.5969  95.0
 31.6263   5.0
 31.6965  54.0
 31.7892  58.0
 31.82    74.0
  ⋮           
 36.4619  62.0
 36.5348  71.0
 36.5596  24.0
 36.7846   2.0
 36.8118  47.0
 36.8214  46.0
 36.9126   7.0
 37.0051  83.0
 37.5293  27.0
 37.5431  63.0
 37.8893  78.0
 38.1315  21.0

In [11]:
## plot for illustration
Best_I = Int(sortslices([[Iteration_Dict[k][1] for k = 1:N_I] collect(1:N_I)],dims = 1)[1,2])
scatter(Pts[:,1], Pts[:,2], label = "Points")
scatter!([Pts[Origin,1]], [Pts[Origin,2]], label = "Origin")
scatter!([Pts[Goal,1]], [Pts[Goal,2]], label = "Goal")

plot!(Pts[Iteration_Dict[Best_I][2],1], Pts[Iteration_Dict[Best_I][2],2], label = "L = $(Iteration_Dict[Best_I][1])")

In [14]:
Iteration_Dict[Best_I]

(28.323442152957444, [2, 16, 195, 151, 61, 69, 94, 33, 156, 22  …  141, 150, 114, 131, 96, 135, 3, 147, 10, 8])

In [12]:
Random_L_Collect = Float64[]
for r = 1:1000
    RandomPath = [[Origin]; sample(setdiff(collect(1:N),[Origin,Goal]), N-2, replace = false);[Goal]]
    Random_L = sum([Dist[RandomPath[i],RandomPath[i+1]] for i=1:N-1])
    push!(Random_L_Collect, Random_L)
end

In [13]:
sort(Random_L_Collect)

1000-element Array{Float64,1}:
  90.39072325187297
  94.85505898471989
  95.14324261951978
  95.3625759205883 
  95.43661788192786
  95.85022828080199
  95.8912779751097 
  96.15585585679584
  96.24890462445956
  96.38535840478666
  96.87836066720705
  97.01557114688174
  97.19780310870674
   ⋮               
 110.20427578505272
 110.22241991984623
 110.30130459829708
 110.51541811418505
 110.66379631635776
 110.83746081707665
 111.16303870967776
 111.17813468746112
 111.2676381169706 
 111.34858088128654
 111.51534984992058
 111.90139872054169

## Combine everything into a function

In [1]:
## This function implements ACO (Ant Colony Optimization) on a matrix of coordinates of points
## Inputs: 
## (1) Pts, N x d; each row represents the coordinates of a point
## (2) Origin = the starting point, Goal = the goal
## (3) N_I = number of iteration; default value = 100
## (4) alp = alpha, bet = beta; these are hyperparameters 
##           controlling the influence of distance and remaining pheromone on the path
## (5) rho = evaporation rate, Q = the total amount of pheromone an ant carries
using LinearAlgebra ## for the function "norm"
using StatsBase ## for function "sample"

function ACO_p(Pts::Array{Float64,2}, Origin::Int, Goal::Int, N_I = 100::Int, alp = 1.0, bet = 1.0, 
        rho = 0.3, Q = 10)
    N = size(Pts,1)
    ## compute the distance matrix
    Dist = [norm(Pts[i,:].-Pts[j,:]) for i=1:N, j=1:N]; ## the distance matrix of the points
    
    ## Dictionary for collecting the best result in each iteration
    Iteration_Dict = Dict()
        
    for I = 1:N_I
        Tau = (1/N).*ones(N,N); ## the prior transition probability
        Best_L = Inf ## the best (shortest) length
        BestPath = 0 ## the best (shortest) path
    
        L = 0 ## variable for stopping criterion
        L_previous = Inf ## variable for stopping criterion
    
        Final_path = 0
        Final_L = Inf
    
        while (abs(L-L_previous)>1.0e-7)
            L_previous = copy(L)
            Cand = setdiff(collect(1:N),union([Origin],[Goal])) ## the candidates
            cp = copy(Origin) ## cp =  current point
            path_a = Int[Origin]
            
            if Origin!=Goal
                for p = 2:N-1
                    w = aweights((Tau[cp,Cand].^alp).*((Dist[cp,Cand].^(-1)).^bet))
                    cp = sample(Cand,w)
                    Cand = setdiff(Cand,[cp])
                    push!(path_a,cp)
                end
                push!(path_a,Goal)
            else
                for p = 2:N
                    w = aweights((Tau[cp,Cand].^alp).*((Dist[cp,Cand].^(-1)).^bet))
                    cp = sample(Cand,w)
                    Cand = setdiff(Cand,[cp])
                    push!(path_a,cp)
                end
                push!(path_a,Goal)                
            end
            path_a
            Final_path = path_a
            L = sum([Dist[path_a[i],path_a[i+1]] for i=1:N-1]) ## total length of path_a
            
            ## println(L)
            
            ## store the best record
            if L<Best_L
                Best_L = L
                BestPath = path_a
            end
    
            Tau = Tau*(1-rho) ## evaporation update
            for i=1:N-1
                Tau[path_a[i], path_a[i+1]] = Tau[path_a[i], path_a[i+1]] + Q/L ## Pheromone update
            end
            Tau
        end
        
        Iteration_Dict[I] = (Best_L, BestPath)
    end
    
    ## Find out the best result among all iterations
    Best_I = Int(sortslices([[Iteration_Dict[k][1] for k = 1:N_I] collect(1:N_I)],dims = 1)[1,2])
    
    return Iteration_Dict[Best_I]
end

ACO_p (generic function with 6 methods)

In [2]:
## This function implements ACO (Ant Colony Optimization) on a matrix of distances
## Inputs: 
## (1) Dist = the distance matrix
## (2) Origin = the starting point, Goal = the goal
## (3) N_I = number of iteration; default value = 100
## (4) alp = alpha, bet = beta; these are hyperparameters 
##           controlling the influence of distance and remaining pheromone on the path
## (5) rho = evaporation rate, Q = the total amount of pheromone an ant carries
function ACO_d(Dist::Array{Float64,2}, Origin::Int, Goal::Int, N_I = 100::Int, alp = 1.0, bet = 1.0, 
        rho = 0.3, Q = 10)
    N = size(Dist,1)
    ## Dictionary for collecting the best result in each iteration
    Iteration_Dict = Dict()
        
    for I = 1:N_I
        Tau = (1/N).*ones(N,N); ## the prior transition probability
        Best_L = Inf ## the best (shortest) length
        BestPath = 0 ## the best (shortest) path
    
        L = 0 ## variable for stopping criterion
        L_previous = Inf ## variable for stopping criterion
    
        Final_path = 0
        Final_L = Inf
    
        while (abs(L-L_previous)>1.0e-7)
            L_previous = copy(L)
            Cand = setdiff(collect(1:N),union([Origin],[Goal])) ## the candidates
            cp = copy(Origin) ## cp =  current point
            path_a = Int[Origin]
            
            if Origin!=Goal
                for p = 2:N-1
                    w = aweights((Tau[cp,Cand].^alp).*((Dist[cp,Cand].^(-1)).^bet))
                    cp = sample(Cand,w)
                    Cand = setdiff(Cand,[cp])
                    push!(path_a,cp)
                end
                push!(path_a,Goal)
            else
                for p = 2:N
                    w = aweights((Tau[cp,Cand].^alp).*((Dist[cp,Cand].^(-1)).^bet))
                    cp = sample(Cand,w)
                    Cand = setdiff(Cand,[cp])
                    push!(path_a,cp)
                end
                push!(path_a,Goal)                
            end
            path_a
            Final_path = path_a
            L = sum([Dist[path_a[i],path_a[i+1]] for i=1:N-1]) ## total length of path_a
            
            ## println(L)
            
            ## store the best record
            if L<Best_L
                Best_L = L
                BestPath = path_a
            end
    
            Tau = Tau*(1-rho) ## evaporation update
            for i=1:N-1
                Tau[path_a[i], path_a[i+1]] = Tau[path_a[i], path_a[i+1]] + Q/L ## Pheromone update
            end
            Tau
        end
        
        Iteration_Dict[I] = (Best_L, BestPath)
    end
    
    ## Find out the best result among all iterations
    Best_I = Int(sortslices([[Iteration_Dict[k][1] for k = 1:N_I] collect(1:N_I)],dims = 1)[1,2])
    
    return Iteration_Dict[Best_I]
end

ACO_d (generic function with 6 methods)

## Test the function

In [5]:
include("ACO_p_and_ACO_d.jl")

ACO_d (generic function with 6 methods)

In [6]:
Pts = rand(100,2);
D = [norm(Pts[i,:]-Pts[j,:]) for i = 1:size(Pts,1), j = 1:size(Pts,1)]

100×100 Array{Float64,2}:
 0.0       0.698181  1.02328   0.442302   …  0.750047   0.851614  0.331752
 0.698181  0.0       0.461539  0.289025      0.525474   0.154039  0.387192
 1.02328   0.461539  0.0       0.715989      0.976673   0.415165  0.695616
 0.442302  0.289025  0.715989  0.0           0.418262   0.431836  0.220414
 0.370743  0.338436  0.735581  0.0775809     0.481739   0.488258  0.154329
 0.526899  0.406287  0.523525  0.411116   …  0.819819   0.529435  0.245084
 0.38116   0.593814  1.03193   0.315938      0.397375   0.72051   0.426791
 0.584119  0.493529  0.527708  0.513264      0.922316   0.60428   0.336221
 0.918114  0.452894  0.840838  0.503111      0.26001    0.427285  0.715248
 0.253663  0.451294  0.82214   0.189604      0.550726   0.602999  0.150215
 0.42753   0.670816  0.788725  0.572148   …  0.985981   0.807091  0.351818
 0.503314  0.324071  0.778191  0.111422      0.307194   0.44357   0.32891 
 0.723086  0.47366   0.926172  0.372479      0.0521722  0.525258  0.59134 

In [7]:
O = 1 ## O = Origin
G = 2 ## G = Goal
(L,P) = ACO_p(Pts, O, G)

(19.416316382397124, [1, 59, 75, 48, 40, 93, 49, 37, 82, 87  …  79, 22, 14, 51, 41, 27, 28, 45, 84, 2])

In [8]:
using Plots
plotly()

scatter(Pts[:,1], Pts[:,2], label = "Points")
scatter!([Pts[O,1]], [Pts[O,2]], label = "Origin")
scatter!([Pts[G,1]], [Pts[G,2]], label = "Goal")
plot!(Pts[P,1], Pts[P,2], label = "Shortest Path, Length = $(round(L,digits = 2))")

In [9]:
O = 3 ## O = Origin
G = 3 ## G = Goal
(L,P) = ACO_d(D, O, G)

(17.96063471650791, [3, 36, 67, 14, 22, 38, 34, 46, 35, 20  …  16, 93, 51, 28, 95, 79, 49, 11, 31, 3])

In [10]:
using Plots
plotly()

scatter(Pts[:,1], Pts[:,2], label = "Points")
scatter!([Pts[O,1]], [Pts[O,2]], label = "Origin")
scatter!([Pts[G,1]], [Pts[G,2]], label = "Goal")
plot!(Pts[P,1], Pts[P,2], label = "Shortest Path, Length = $(round(L,digits = 2))")